In [5]:
#We first import numpy as a math package, networkx to work with graphs and matplotlib to create plots.
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [8]:
#This function pseudorandomly generates a graph, with standard randomseed 1(can be changed by giving random seed as third argument).
#Input int n (number of vertices), float prob(probability of there being a connection), randomseed=1 (value for our seudorandomgenerator to consistently generate the same random graph)
#Returns int len(vertices), which is the total amount of vertices in the graph, and list edgeset containing all edges of the random graph.
def genRandomGraph(n,prob,randomseed = 1):
    #Generate the vertex set based on the given number of vertices
    vertices = []
    for i in range(0,n):
            vertices.append(i)
    #Create a pseudorandom edgeset corresponding to the vertexset based on given probability and number of vertices.
    edgeset = []
    np.random.seed(randomseed)
    nsquared = n*n
    ncubed = n*n*n
    #Create enough zeroes and ones corresponding to all possible edges in the list random_connections
    random_connections = np.random.binomial(1,prob,size = ncubed)
    #
    for i in range(0,len(vertices)):
        for j in range(i+1, len(vertices)):
            if (int(random_connections[i*nsquared + j]) == 1):
                edgeset.append((i, j))
    
    return [len(vertices), edgeset]

In [10]:
#This function generates a graph based on gives verices and edges and tests if this graph is connected.
#Input int numberOfVertices, list edges
#Returns boolean connectedBoolean
def isConnected(numberOfVertices, edges):
    #create empty Graph G with networkx
    G = nx.Graph()
    #Add vertices to Graph G based on input numberOfVertices
    for vertex in range(0,numberOfVertices):
        G.add_node(vertex)
    #Add edges to graph G based on input edges
    for edge in edges:
        G.add_edge(edge[0],edge[1])
    #check if graph G is connected, store result as Boolean and return it.
    connectedBoolean = nx.is_connected(G)
    return connectedBoolean

In [11]:
#This function finds the connected components of a grapj.
#Input int aantalVertices, list edges
#Return connected components of a graph G based on the given vertices and edges.
def generateComponents(numberOfVertices, edges):
    G = nx.Graph()
    for vertex in range(0,numberOfVertices):
        G.add_node(vertex)
    for edge in edges:
        G.add_edge(edge[0],edge[1])
    components = nx.connected_components(G)
    return components

In [12]:
#This function creates a connected graph based on the function genRandomGraph
#Input int n(number of vertices), float prob (average probability that a random possible edge is in the graph)
#Returns list containing (number of vertices, list of edges)
def generateConnectedGraph(n, prob):
    random_seed = 0
    random_graph = genRandomGraph(n, prob, random_seed)
    connectedBoolean = False
    while connectedBoolean == False:
        random_seed += 1
        random_graph = genRandomGraph(n,prob,random_seed)
        connectedBoolean = isConnected(random_graph[0], random_graph[1])
    print("Random seed = " + str(random_seed))
    return (random_graph)
    

In [1]:
#This function takes a graph, a k-coloring, and ordering and the vertices of degree 0 and used these to find a corresponding ideal and Groebner basis.
#Input list graphInfo, int k, str ordering, list verticesWithoutEdges
#Returns Ideal I, Groebner basis G
def main(graphInfo, k, ordering):
    numberOfVertices = graphInfo[0]
    #Create a Polynomialring based on the cyclotomic polynomial and number field.
    R.<w> = PolynomialRing(QQ)
    b = [0,w-1, w+1, w^(2)+w+1, w^(2)+1, w^(4)+w^(3)+w^(2)+w+1, w^(2)-w+1, w^(6)+w^(5)+w^(4)+w^(3)+w^(2)+w+1, w^(4)+1, w^(6)+w^(3)+1]
    cyclitomicPolynomial = b[k]
    F.<w> = NumberField(cyclitomicPolynomial)
    R = PolynomialRing(F,numberOfVertices, names='x', order=ordering)
    
    
    edges = graphInfo[1]
    x = R.gens()
    index = list(range(0,n))
    functions = []
    #add equations corresponding to constraints for coloring
    for i in index:
        functions.append(x[i]^k - 1)
    
    for edge in edges:
        functions.append(sum([x[edge[0]]^(k-1-m) * x[edge[1]]^m for m in range(0,k)]))
    
    substitutionDictionairy = {}
    
    #color the first vertex with the first of k-colors, then color an adjacent vertex with the second out of k colors.   
    firstConnectedVertex = edges[0][0]
    secondConnectedVertex = edges[0][1]
    
    substitutionDictionairy[x[firstConnectedVertex]] = 1
    substitutionDictionairy[x[secondConnectedVertex]] = w
    
    #substitute the functions into the functions listas well, followed by the functions needed to color vertices of degree 0 and the first two connected vertices.
    functions = [f.subs(substitutionDictionairy) for f in functions]    
    
    
    functions.append(x[firstConnectedVertex] - 1)
    functions.append(x[secondConnectedVertex] - w)
    
    #reate ideal based on given functions and a corresponding Groebner basis, then return them.
    I = Ideal(functions)
    G = I.groebner_basis()
    
    return (I,G)

In [52]:
#Determine the number of vertices and the probabilty of a random edge being in the graph.
n = 14
p = 0.4

#Create connected graph based on above n and p
graphInfo = generateConnectedGraph(n,p)
print(graphInfo)



Random seed = 1
[14, [(0, 1), (0, 11), (0, 13), (1, 2), (1, 3), (1, 4), (1, 6), (1, 7), (1, 10), (1, 12), (1, 13), (2, 3), (2, 5), (2, 8), (2, 9), (2, 11), (2, 13), (3, 4), (3, 12), (3, 13), (4, 5), (4, 9), (4, 11), (5, 8), (5, 9), (5, 10), (6, 8), (6, 10), (6, 11), (6, 13), (7, 11), (7, 12), (8, 9), (8, 12), (9, 10), (9, 11), (9, 12), (10, 13), (11, 12), (12, 13)]]
[]


In [55]:
#Set k = 1, fix an ordering. We start ith a Gröbner Basis = [1] i.e. no solution. Then we will run a loop that ends when there is a solution to the k colouring problem that updates k.
k=1
ordering = 'lex'
groebnerBasis = [1]
while groebnerBasis == [1]:
    k += 1
    print(k)
    (I, groebnerBasis) = main(graphInfo, k, ordering)
    
print('The chromatic number of the graph is: ', k)

2
3
4


The chromatic number of the graph is:  4
